# Data processing

In [10]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import random
import re 
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

In [37]:
# Tokenize a sentence
def clean_str(string, tolower=True):
    """
    Tokenization/string cleaning.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    if tolower:
        string = string.lower()
    return string.strip()


# reads the content of the file passed as an argument.
# if limit > 0, this function will return only the first "limit" sentences in the file.
def loadTexts(filename, limit=-1):
    dataset=[]
    with open(filename) as f:
        line = f.readline()
        cpt=1
        skip=0
        while line :
            cleanline = clean_str(f.readline()).split()

            
            if cleanline: 
                dataset.append(cleanline)
            else: 
                line = f.readline()
                skip+=1
                continue
            if limit > 0 and cpt >= limit: 
                break
            line = f.readline()
            cpt+=1        

        print("Load ", cpt, " lines from ", filename , " / ", skip ," lines discarded")
        
    for i in range(len(dataset)):
        dataset[i] = dataset[i][:-2] # remove the label at the end of each sentence
        if dataset[i]==[',']:
            dataset[i]=[]
    

    return dataset



def make_loader(data, labels, batch_size=32):
    X = th.stack(data)      # (N, L)
    y = labels              # (N,)
    dataset = TensorDataset(X, y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


<>:16: SyntaxWarning: invalid escape sequence '\('
<>:17: SyntaxWarning: invalid escape sequence '\)'
<>:18: SyntaxWarning: invalid escape sequence '\?'
<>:16: SyntaxWarning: invalid escape sequence '\('
<>:17: SyntaxWarning: invalid escape sequence '\)'
<>:18: SyntaxWarning: invalid escape sequence '\?'
C:\Users\maloc\AppData\Local\Temp\ipykernel_5832\3242241957.py:16: SyntaxWarning: invalid escape sequence '\('
  string = re.sub(r"\(", " \( ", string)
C:\Users\maloc\AppData\Local\Temp\ipykernel_5832\3242241957.py:17: SyntaxWarning: invalid escape sequence '\)'
  string = re.sub(r"\)", " \) ", string)
C:\Users\maloc\AppData\Local\Temp\ipykernel_5832\3242241957.py:18: SyntaxWarning: invalid escape sequence '\?'
  string = re.sub(r"\?", " \? ", string)


In [38]:
train_data = loadTexts("train.tsv")
test_data = loadTexts("test.tsv")
dev_data = loadTexts("dev.tsv")

for i in range(len(dev_data)):
    dev_data[i] = dev_data[i][:-2] # remove the "," and other index at the end of each sentence


Load  21706  lines from  train.tsv  /  0  lines discarded
Load  2714  lines from  test.tsv  /  1  lines discarded
Load  2714  lines from  dev.tsv  /  0  lines discarded


In [39]:
print(dev_data[0:20])

[['you', 'know', 'the', 'answer', 'man', ',', 'you', 'are', 'programmed', 'to', 'capture', 'those', 'codes', 'they', 'send', 'you', ',', 'don', 't', 'avoid', 'them', '!'], ['the', 'economy', 'is', 'heavily', 'controlled', 'and', 'subsidized', 'by', 'the', 'government', 'in', 'any', 'case', ',', 'i', 'was', 'poking', 'at', 'the', 'lack', 'of', 'nuance', 'in', 'us', 'politics', 'today'], ['thank', 'you', 'for', 'your', 'vote', 'of', 'confidence', ',', 'but', 'we', 'statistically', 'ca', "n't", 'get', 'to'], ['there', 'it'], ['good', 'we', 'do', "n't", 'want', 'more', 'thrash', 'liberal', 'offspring', 'in'], ['i', 'went', 'to', 'a', 'destination', 'wedding', 'being', 'the', 'only', 'single', 'person', 'promised', 'to', 'never', 'put', 'myself', 'in', 'that', 'situation', 'again'], ['like', 'this', 'just', 'cuz', 'of', 'the', 'name', 'rhymes', 'background', 'raps', 'but', 'dude', 'your', 'name', 'is', 'sick'], ['as', 'an', 'anesthesia', 'resident', 'this', 'made', 'me', 'blow', 'air', 'out

### Word dictionnary

In [41]:
class WordDict:
    # constructor, words must be a set containing all words
    def __init__(self, words):
        assert type(words) == set
        # TODO
        self.words=sorted(list(words))
        self.word_dict_={w: i for i, w in enumerate(self.words)}
        self.id_dict_={i: w for i, w in enumerate(self.words)}
        
        # return the integer associated with a word
    def word_to_id(self, word):
        assert type(word) == str
        return self.word_dict_['word']
        # TODO

        # return the word associated with an integer
    def id_to_word(self, idx):
        assert type(idx) == int
        # TODO
        return self.id_dict_['idx']
    
        # number of word in the dictionnary
    def __len__(self):
        # TODO
        return len(self.word_dict_)


        
train_words = set()
for sentence in train_data:
    train_words.update(sentence)
    train_words.update(["<bos>", "<eos>"])

word_dict = WordDict(train_words)

len(word_dict)


18657

### Tensoring the data

# N Gram model

In [ ]:
class Ngram:
    def __init__(self,n):
        self.n=n
        self.word_count={}
        pass

    def create_ngrams(self,sentence):
        words = sentence.split()
        ngrams = {}
        for i in range(len(words) - self.n + 1):
            ngram = tuple(words[i:i + self.n])
            if ngram not in ngrams:
                ngrams[ngram] = 0
            ngrams[ngram] += 1

        return ngrams
    
    def 
    


# LSTM model